# Import data to sgkit

In [1]:
%run setup.ipynb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dask.diagnostics import ProgressBar
from sgkit.io.vcfzarr_reader import vcfzarr_to_zarr

In [4]:
input = here() / 'data/external/ag1000g/phase2/AR1/variation/main/zarr/pass/ag1000g.phase2.ar1.pass/'
output = here() / 'data/sgkit/ag1000g.zarr'

In [5]:
with ProgressBar():
    vcfzarr_to_zarr(input, output, grouped_by_contig=True, consolidated=True)

[########################################] | 100% Completed |  5.4s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 32.8s
[########################################] | 100% Completed |  7.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 42.4s
[########################################] | 100% Completed |  5.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 31.9s
[########################################] | 100% Completed |  6.6s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 38.4s
[########################################] | 100% Completed |  2.7s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [5]:
import xarray as xr

In [6]:
ds = xr.open_zarr(str(output), concat_characters=False)
ds

,Array,Chunk
Bytes,132.10 GB,63.96 MB
Shape,"(57837885, 1142, 2)","(524288, 61, 2)"
Count,2110 Tasks,2109 Chunks
Type,int8,numpy.ndarray
,Array,Chunk
Bytes,132.10 GB,63.96 MB
Shape,"(57837885, 1142, 2)","(524288, 61, 2)"
Count,2110 Tasks,2109 Chunks
Type,bool,numpy.ndarray
,Array,Chunk


Rechunk with `rechunk_bgen` (hacked to rechunk `call_genotype` vars)

In [7]:
from sgkit.io.bgen import rechunk_bgen

In [8]:
output_rechunked = here() / 'data/sgkit/ag1000g_rechunked.zarr'

In [9]:
with ProgressBar():
    ds_rechunked = rechunk_bgen(ds, output_rechunked, chunk_length=524288, chunk_width=1142, max_mem="2GB", compressor=zarr.Blosc(cname="lz4", clevel=5, shuffle=2), probability_dtype=None, pack=False)

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 23min 50.7s


In [34]:
source_group = zarr.open(str(output))
source = source_group["call_genotype"]
source.info

Name,/call_genotype
Type,zarr.core.Array
Data type,int8
Shape,"(57837885, 1142, 2)"
Chunk shape,"(524288, 61, 2)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,132101729340 (123.0G)
No. bytes stored,7171639896 (6.7G)


Why does doing it directly only take 2 mins?

In [35]:
target_chunks = (524288, 1142, 2)
max_mem = '2GB'

target_store = str(here() / 'data/sgkit/ag1000g_rechunked2.zarr')
temp_store = str(here() / 'data/sgkit/ag1000g_rechunked2_tmp.zarr')

In [36]:
from rechunker import api as rechunker_api
plan = rechunker_api.rechunk(source, target_chunks, max_mem, target_store, temp_store=temp_store)
plan

<Rechunked>
* Source      : <zarr.core.Array '/call_genotype' (57837885, 1142, 2) int8>

* Target      : <zarr.core.Array (57837885, 1142, 2) int8>

In [37]:
with ProgressBar():
    plan.execute()

[########################################] | 100% Completed |  1min 48.4s
